# 딥 러닝
## 3-4 ResNet을 활용한 이미지 분류
### 3-4-1 ResNet을 활용한 이미지 분류의 개요
ResNet을 이용해 이미지 분류를 수행한다. 데이터셋은 `CIFAR10`을 사용하며 이미지를 10개 클래스로 분류한다.
### 3-4-2 ResNet이란?
ResNet에서는 Residual block이라는 short-cut 구조를 만들어 레이어를 깊게 만들며 복잡한 특징을 추출하는 동시에 성능 약화를 막았다.
- CNN과 Residal block

shortcut connection이라고 불리는 우회 경로를 추가하여 학습이 필요하지 않은 경우 우회하여 깊은 레이어를 사용한 학습이 가능하다.

- Plain architecture와 Bottleneck architecture

`Plain architecture` 는 3X3 사이즈인 동일 매수의 커널을 가진 컨볼루셔널 레이어를 나란히 2개 사용한 것이다. ResNet 논문에서의 최소 케이스에서는 3X3의 커널 64개를 가진 conv layer를 나란히 2개 연결하여 사용한다. <br>
<br>
`Bottleneck architecture`는 레이어를 하나 더 가진다. 1X1 사이즈와 3X3 사이즈 커널의 개수가 동일한 conv layer 2개를 사용해 출력의 차원을 줄이고, 1X1 사이즈 커널의 수의 4배의 커널을 가진 conv layer에서 차원을 복원하기 때문에 `Bottleneck`이라는 이름이 붙여졌다.<br>
ResNet 논문에서의 최소 케이스에서는 1X1 사이즈 커널 64개를 가진 conv layer, 3X3 사이즈 커널 64개를 가진 conv layer, 1X1 사이즈 커널 256개를 가진 conv layer를 사용한다.
<br><br>
자세한 사항은 <br>
[Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)<br>
[Identity Mappings in Deep Residual Networks](https://arxiv.org/abs/1603.05027) <br>
에서 확인 가능하다.
### 3-4-3 패키지 임포트

In [1]:
# 패키지 임포트
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Activation, Add, BatchNormalization, Conv2D, Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

2022-01-26 13:03:54.591299: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-26 13:03:54.591322: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### 3-4-4 데이터 셋 준비 및 확인 & 3-4-5 데이터 셋 전처리 및 확인
훈련 이미지와 테스트 이미지의 표준화를 위해 `ImageDataGenerator`를 사용한다. 훈련 라벨과 테스트 라벨은 `one-hot encoding`으로 전처리한다.

In [2]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# 데이터 셋 전처리
train_images = train_images
train_labels = to_categorical(train_labels, 10)
test_images = test_images
test_labels = to_categorical(test_labels, 10)

# 데이터 셋 전처리 후 형태 확인
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(50000, 32, 32, 3)
(50000, 10)
(10000, 32, 32, 3)
(10000, 10)


### 3-4-6 Functional API 이용
복잡한 모델을 정의하기 위한 인터페이스이다.<br>
- Input : 입력 데이터 형태는 별도의 `Input` 클래스에서 지정한다.
- 레이어 입출력 : Dense의 우측에 붙어있는 `(Input)`이나 `(x)`는 레이어로의 입력이다. Dense좌측에 붙어있는 `x=`은 레이어로부터의 출력이다. 
- Model : 마지막에 `Model` 인스턴스를 생성한다. 인수에는 네트워크 전체 입력과 출력을 지정한다.

### 3-4-7 모델 생성
- Convolutional layer 생성
> kernel_initializer에는 커널 가중치 행렬의 초기값이 들어간다. `he_normal`은 정규분포에 따른 초기값이다. <br>
> kernel_regularizer에는 커널 가중치에 적용할 표준화를 지정한다.<br>
> padding에는 입력과 같은 사이즈로 되돌릴 때에는 'same', 아니면 'valid'를 입력한다.

- 정규화

Regularization은 모델을 복잡하게 하는 가중치에 가중치 양만큼 페널티를 부여하여 모델이 복잡해지지 않도록 하는 방법이다.

- Residual block 생성

convolutional layer의 `커널 수, 커널 사이즈, 스트라이드`를 의미하는 `tuple`들로 bottleneck architecture를 구성한다. <br>
숏컷 커넥션의 위치가 다른 2종류의 보틀넥 아키텍쳐를 총 54개 연결했다. (레지듀얼 블록 A, 레지듀얼 블록 B, 서로 숏컷 커넥션의 시작 위치가 각각 다르다)<br>
<br>

> Residual block A <br>
> BatchNormalization - ReLU -(여기에 시작 위치)- conv layer - BN - ReLU - conv layer - BN - ReLU - Conv layer - Add

> Residual block B <br>
> 여기에 시작 위치 - BatchNormalization - ReLU - conv layer - BN - ReLU - conv layer - BN - ReLU - Conv layer - Add

`BatchNoramlization(BN)`은 학습을 안정시켜 학습 속도를 높이는 방법의 한 가지로, 배치 표준화라고 부르며 `conv layer`와 `activation fn` 사이에 추가한다. <br>
보통 `Dropout`보다 성능이 좋으며 혼용은 하지 않는 것이 좋다.

- 모델 네트워크 구조

`conv layer` 뒤에 `Residual block` 54개와 `pooling layer`를 추가하며 이 부분에서 특징을 추출한다.<br>
가장 마지막에 `FC layer`를 하나 추가한다. `GlobalAveragePooling2D`의 출력이 1차원이므로 Flatten할 필요는 없다. 이 부분에서 분류를 수행한다.

In [9]:
# convolutional layer 생성
def conv(filters, kernel_size, strides=1):
    return Conv2D(filters, kernel_size, strides=strides, padding='same', use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))

# Residual block A 생성
def first_residual_unit(filters, strides):
    def f(x):
        # BN -> ReLU
        x = BatchNormalization()(x)
        b = Activation('relu')(x)
        
        # Conv layer -> BN -> ReLU
        x = conv(filters // 4, 1, strides)(b)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        
        # Conv layer -> BN -> ReLU
        x = conv(filters // 4, 3)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        
        # Conv layer ->
        x = conv(filters, 1)(x)
        
        # short-cut 사이즈 조정
        sc = conv(filters, 1, strides)(b)
        
        # Add
        return Add()([x, sc])
    return f

# Residual block B 생성
def residual_unit(filters):
    def f(x):
        sc = x
        # BN -> ReLU
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        
        # Conv layer -> BN -> ReLU
        x = conv(filters // 4, 1)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        
        # Conv layer -> BN -> ReLU
        x = conv(filters // 4, 3)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        
        # Conv layer ->
        x = conv(filters, 1)(x)
        
        # Add
        return Add()([x, sc])
    return f

# Residual A, Residual B X 17 생성
def residual_block(filters, strides, unit_size):
    def f(x):
        x = first_residual_unit(filters, strides)(x)
        for i in range(unit_size - 1):
            x = residual_unit(filters)(x)
        return x
    return f

In [10]:
# 입력 데이터 형태
input = Input(shape=(32, 32, 3))

# Conv layer
x = conv(16, 3)(input)

# Residual block X 54
x = residual_block(64, 1, 18)(x)
x = residual_block(128, 2, 18)(x)
x = residual_block(256, 2, 18)(x)

# -> BN -> ReLU
x = BatchNormalization()(x)
x = Activation('relu')(x)

# pooling layer
x = GlobalAveragePooling2D()(x)

# FC layer
output = Dense(10, activation='softmax', kernel_regularizer=l2(0.0001))(x)

# 모델 생성
model = Model(inputs=input, outputs=output)

### 3-4-8 컴파일
손실함수는 `categorical_crossentropy`, 최적화 함수는 `SGD`, 평가 지표는 `acc`으로 지정한다.

### 3-4-9 ImageDataGenerator 준비
`ImageDataGenerator`는 데이터 셋 이미지의 정규화와 augmentation(부풀리기)를 수행하는 클래스다. <br>
`featurewise_center`(데이터 셋 전체의 입력 평균 0), `featurewise_std_normalization`(입력을 데이터셋 표준편차로 표준화)을 이용한 정규화를 시킨다. <br>
`width_shift_range`(랜덤 수평 이동), `height_shift_range`(랜덤 수직 이동), `horizontal_flip`(수평 방향 입력 무작위 반전)을 이용해 부풀린다. <br>

In [12]:
# 컴파일
model.compile(loss='categorical_crossentropy', optimizer=SGD(momentum=0.9), metrics=['acc'])

# ImageDataGenerator 준비
train_gen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True
)
test_gen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization=True
)

# 데이터 셋 전체 총합량을 미리 계산
for data in (train_gen, test_gen):
    data.fit(train_images)

### 3-4-10 LearningRateScheduler 준비
`LearningRateScheduler`는 학습 중 `학습률`을 변화시키는 콜백이다. epoch을 인수로 입력받아 학습률을 반환하는 함수를 생성하고, 이를 LearningRateScheduler에 인수로 전달한다. 이를 `fit()` 또는 `fit_generator()`의 인수로 지정하면 지정한 학습률을 적용할 수 있다.

In [13]:
# LearningRateScheduler 준비
def step_decay(epoch):
    x = 0.1,
    if epoch >= 80: x = 0.01
    if epoch >= 120: x = 0.001
    return x
lr_decay = LearningRateScheduler(step_decay)

### 3-4-11 학습
`fit_generator()`를 사용하여 `ImageDataGenerator`를 학습한다. 

### 3-4-12 모델 저장 & 3-4-13 그래프 표시 & 3-4-14 평가
모델을 파일로 저장후, `fit()`의 반환값인 `history`를 matplotlib으로 그래프로 나타낸다. <br>
그 이후에 `evaluate_generator()`를 이용하여 평가를 수행하고 정답률을 얻는다. <br>

### 3-4-15 추론 
첫 번째 10개의 테스트 이미지에 대한 추론을 수행하고 예측 결과를 얻는다. <br>
`ImageDataGenerator`의 예측에는 `predict_generator()`를 사용한다.

In [ ]:
# 학습
batch_size = 128
history = model.fit_generator(
    train_gen.flow(
        train_images,
        train_labels,
        batch_size = batch_size
    ),
    epochs=200,
    steps_per_epoch = train_images.shape[0] // batch_size,
    validation_data = test_gen.flow(
        test_images,
        test_labels,
        batch_size = batch_size
    ),
    validation_steps = test_images.shape[0] // batch_size,
    callbacks=[lr_decay]
)

# 모델 저장
model.save('resnet.h5')

# 그래프 표시
plt.plot(history.history['acc'], label='acc')
plt.plot(history.history['val_acc'], label='val_acc')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(loc='best')
plt.show()

# 평가
batch_size = 128
test_loss, test_acc = model.evaluate_generator(
    test_gen.flow(test_images, test_labels, batch_size = batch_size),
    steps=10
)
print('loss: {:.3f}\nacc: {:.3f}'.format(test_loss, test_acc))

In [ ]:
# 추론할 이미지 표시
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(test_images[i])
plt.show()

# 추론한 라벨 표시
test_predictions = model.predict_generator(
    test_gen.flow(test_images[0:10], shuffle=False, batch_size=1), steps=10
)
test_predictions = np.argmax(test_predictions, axis=1)
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
print([labels[n] for n in test_predictions])